In [ ]:
""" This is a heavily documented notebook with exploratory data analyses for Telegram Channel Archive project
https://github.com/githubbar/telegram_channel_archive


For working with the SQLite backend we are choosing Ibis over Pandas for data analysis because it does execuste a lot of stuff at the backend and is therefore more suitable for large databases.
see: https://voltrondata.com/blog/ibis-explained-making-dataframes-big-and-small-more-delightful
""" 

# TEMP: conda activate /N/slate/oleykin/.conda/envs/tele
# TEMP: conda list -e > requirements.txt

"""Connect to SQLite DB"""
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
import numpy as np
import ibis

ibis.options.interactive = True
ibis.options.repr.interactive.max_rows = 20

con = ibis.sqlite.connect('db.sqlite')
# con.list_tables()

channel  = con.table('channel')
msg = con.table('message')


In [ ]:
# >>>>>> Check start/end date for each channel 
import datetime
from datetime import timedelta
import duckdb
from ibis import _
t = msg.join(channel, msg.channel_id == channel.id) 
print('Date ranges for 2022')
print(t.filter(_.date.cast("timestamp").year() == 2022).group_by('title').aggregate(
    min_time=_.date.min().cast("timestamp"), 
    max_time=_.date.max().cast("timestamp"),
)
)
# group by channel name
print('Date ranges for 2023')
print(t.filter(_.date.cast("timestamp").year() == 2023).group_by('title').aggregate(
    min_time=_.date.min().cast("timestamp"), 
    max_time=_.date.max().cast("timestamp"),
)
)

# TODO: timestamp diff not implemented in Ibis?
# t = t.mutate(
#     days_diff = _.max_time.delta(datetime.datetime.now(), 'day')    
# )


In [ ]:
# >>>>>> Look at nulls (gap analysis); make sure they make sense.
import datetime
from datetime import timedelta
from ibis import _

t = msg.join(channel, msg.channel_id == channel.id)
t = t.filter(_.last_edit_date == None).select(_.title, _.text, _.total_views, _.last_edit_date)
print(f'Found {t.title.count()} records with the field last_edit_date == NULL. These are mostly pinned posts')
# t = t.filter(_.text != '')
print(f'But some {t.title.count()} are not, because they have text in them')
# Looks like an forward announcement for the next post, e.g. https://t.me/femagainstwar/237 and https://t.me/femagainstwar/238
# or a group of images


In [ ]:
# >>>>>> Posting frequencies (own posts vs forwards; normalize by .. number of posts in channel/subs?)
import datetime
from rich import print
from datetime import timedelta
from ibis import _
import pandas as pd

# Identify three 100 day periods: one for each target channel
PERIOD_DAYS = 100
periods = {
1166398892 : [[datetime.datetime(2022, 2, 25, 0, 0)], [datetime.datetime(2023, 2, 25, 0, 0)]],
1724215937 : [[datetime.datetime(2022, 3, 15, 0, 0)], [datetime.datetime(2023, 3, 15, 0, 0)]],
1744097497 : [[datetime.datetime(2022, 2, 27, 0, 0)], [datetime.datetime(2023, 2, 27, 0, 0)]]
}
for key in periods:
    periods[key] = [(start[0], start[0] + datetime.timedelta(days=PERIOD_DAYS)) for start in periods[key]]


t = msg.left_join(channel, msg.channel_id == channel.id)
# t1 = t.filter((_.fwd_username != None)).select(_.title, _.text, _.total_views, _.last_edit_date)
# print(f'Found {t1.title.count()} user forwards')
# t2 = t.filter((_.fwd_channel_id != None)).select(_.title, _.text, _.total_views, _.last_edit_date)
# print(f'Found {t2.title.count()} channel forwards')
# Remove forwards
tNoforwards = t.filter((_.fwd_channel_id == None) & (_.fwd_username == None) & (_.text != None))

print('Post frequency per channel (forwards and pins removed)')
dataByChannel = tNoforwards.group_by('title').aggregate(posts_per_day = _.count()/(PERIOD_DAYS*2)).to_pandas()
print(dataByChannel)
# t.to_csv('peroid_freq.csv')

print('Post frequency per period (forwards and pins removed)')
dataByPeriod = []
for key in periods:
    s1 = tNoforwards.count((_.channel_id == key) & (_.date.cast("timestamp").between(periods[key][0][0], periods[key][0][1]))) \
        / (periods[key][0][1] - periods[key][0][0]).days
    s2 = tNoforwards.count((_.channel_id == key) & (_.date.cast("timestamp").between(periods[key][1][0], periods[key][1][1]))) \
        / (periods[key][1][1] - periods[key][1][0]).days
    ttl = channel.filter(_.id == key).title.to_pandas()[0]
    dataByPeriod.append({"channel": ttl, "period1" : s1.to_pandas(), "period2" : s2.to_pandas()})
dataByPeriod = pd.DataFrame.from_records(dataByPeriod)
print(dataByPeriod)
# t.to_csv('peroid_freq.csv')

In [ ]:
# >>>>>>  Altair charts 
# group by channel name
# chart = (
#     alt.Chart(t.group_by("title").aggregate(count=_.count()))
#     .mark_bar()
#     .encode(
#         x="title",
#         y="count",
#         tooltip=["title", "count"],
#     )
#     .properties(width=1024, height=600)
#     .interactive()
# )
# TODO tihs does it by time, do by date (bad interface)
# date.histogram(200)
# t = tDisplay.date.histogram(binwidth=3600.0).cast("timestamp").name('bucket')
# t = tNoforwards.filter(t.title == "Медиа Партизаны | Нет войне").select('date')
# chart = (
#     alt.Chart(tDisplay.date.histogram(nbins=10))
#     .mark_line()
#     .encode(
#         x=alt.X('date:T'),
#         y='count()'
#     )
#     .properties(width=1024, height=600)
#     .interactive()
# )

# chart

In [48]:
# >>>>>> Export posts with media info

import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
import numpy as np
import ibis
from ibis import _
import ibis.selectors as s

ibis.options.interactive = True
ibis.options.repr.interactive.max_rows = 20
"""
con = ibis.sqlite.connect('db.sqlite')
channel  = con.table('channel')
msg = con.table('message')
media = con.table('media')
"""
t = msg.join(channel, msg.channel_id == channel.id) # drop id_right column
t = t.mutate(fwd_title=t['fwd_title'].cast('str'))
t = t.mutate(fwd_username=t['fwd_username'].cast('str'))
t = t.mutate(fwd_channel_id=t['fwd_channel_id'].cast('str'))
# t = t.limit(100)

# Add isPin pin posts 
t = t.mutate(isPin =(_.text == None))

# Add isForward column
t = t.mutate(isForward=(_.fwd_channel_id != None) | (_.fwd_username != None))

# Add count columns for each media type
type_list = media.distinct(on='type')['type'].to_pandas().to_list()
for mediaType in type_list:
    m = media.filter(_.type==mediaType).group_by(["channel_id", "message_id"]).agg(_.count().name(f'{mediaType}_count'))
    # NOTE: left join below to have NULL if no media count and 1 or more otherwise
    t = t.left_join(m, [t.channel_id == m.channel_id, t.id == m.message_id])
    t = t.select(~s.matches('channel_id_right') & ~s.matches('message_id'))

# Drop all unused columns
cols = [
    'id',
    'date',
    'text',
    # 'mentions',
    'total_views',
    'total_fwds',
    'hidden_edit',
    'last_edit_date',
    # 'scheduled',
    # 'via_bot_id',
    # 'noforwards',
    # 'ttl_period',
    'reactions',
    'fwd_title',
    'fwd_username',
    'fwd_channel_id',
    'title',
    'isPin',
    'isForward'
]
[cols.append(f'{mediaType}_count') for mediaType in type_list]

t = t.select(cols)

t.to_csv('reports/data.csv')